In [259]:
from pandas import Series, DataFrame
import pandas as pd
from sklearn import metrics

import nltk
from nltk import word_tokenize
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
%pylab inline


import pandas as pd
import numpy as np
import re


Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [260]:
from scipy.sparse import coo_matrix

### replace related words to "Aspiration"

In [272]:
replaceDict = pd.read_csv("/Users/anqihuang/Desktop/MSBASPRING/aspire_replace.csv")
replaceDict.columns = ['Replace', 'Search']
replaceDict['Search'] = replaceDict['Search'].str.lower()
replaceDict['Search'] = replaceDict['Search'].str.replace("\xc2\xa0", "")

replaceDict['Replace'] = replaceDict['Replace'].str.lower()
attribute = replaceDict['Replace'].unique().tolist()
        
replaceDict = replaceDict.set_index('Search').to_dict()
replaceDict = replaceDict['Replace']
replaceDict

{'aim': 'aspiration',
 'ambition': 'aspiration',
 'desire': 'aspiration',
 'dream': 'aspiration',
 'eager': 'aspiration',
 'goal': 'aspiration',
 'hope': 'aspiration',
 'ideal': 'aspiration',
 'love': 'aspiration',
 'target': 'aspiration',
 'want': 'aspiration',
 'wish': 'aspiration'}

In [273]:
model = pd.read_csv("/Users/anqihuang/Desktop/MSBASPRING/brand_name.csv")
model.columns = ['Model','Replacement']

brands = model['Model'].unique().tolist()
aspiration = ['dream', 'goal', 'target', 'hope','aim','ideal','ambition', 'desire','wish','want','eager','love']

In [289]:
def dictReplace(x):
    for i,j in replaceDict.items():
        my_regex = r"\s" + re.escape(i) + r"\b"
        if re.search(my_regex, x, re.IGNORECASE):
            x = x.replace(i,j)
    return x

edmunds = pd.read_csv("/Users/anqihuang/Desktop/MSBASPRING/after_mapping3.csv",sep="\t",skiprows=[0],encoding='utf-8',header=None)


edmunds.columns=['date','user','text']
edmunds['text'] = edmunds['text'].str.lower()
edmunds.text.ix[3].encode('utf-8')

"i understand it's not about the speed.  i really don't care if the car won't do over 100 as i rarely break the toyota mark.  what i do care about is power and with the manual mode on the auto, i can get it when i want it.  will i buy an auto-equipped g?  in all likelihood, no.  i desperately want to drive the 6 speed."

In [290]:
edmunds['text'] = edmunds['text'].map(dictReplace)
edmunds.text.ix[3].encode('utf-8')

"i understand it's not about the speed.  i really don't care if the car won't do over 100 as i rarely break the toyota mark.  what i do care about is power and with the manual mode on the auto, i can get it when i aspiration it.  will i buy an auto-equipped g?  in all likelihood, no.  i desperately aspiration to drive the 6 speed."

### Find the most 100 frequent appeareds word in all comments 

In [291]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

jobdes_cleaned = []
jobdes_lemmatized = []
jobdes_nostopwords = []

# remove it if you need punctuation
stop_words.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}','****','...','``']) 
comment2 = edmunds["text"].tolist()
comment = "".join(comment2)
word_tokenize = nltk.word_tokenize(comment)
lemma_word = map(lambda x: lemmatizer.lemmatize(x), word_tokenize)
jobdes_lemmatized.append(lemma_word)
no_stop = filter(lambda x: x not in stop_words, word_tokenize)
jobdes_nostopwords.append(no_stop)
cleaned_text = filter(lambda x: x not in stop_words, lemma_word)
jobdes_cleaned.append(cleaned_text)

In [292]:
whole_list = jobdes_cleaned[0]
jobdes_bow_cleaned_freq = nltk.FreqDist(whole_list)
for word, frequency in jobdes_bow_cleaned_freq.most_common(300):
    print('%s;%d' % (word, frequency)).encode('utf-8')

car;6558
bmw;4313
's;4279
n't;3910
wa;3313
acura;3131
'';2556
like;2158
would;2128
one;2040
infiniti;1999
$;1989
ha;1971
audi;1871
get;1790
-;1720
aspiration;1518
think;1479
drive;1347
new;1295
&;1246
year;1231
better;1197
much;1163
doe;1152
lexus;1134
'm;1093
even;1071
good;1064
make;1024
sedan;1012
also;971
driving;962
really;945
look;934
price;933
time;932
know;919
well;917
performance;913
mercedes;913
people;903
luxury;867
cadillac;860
say;825
interior;806
sport;784
could;773
engine;772
honda;769
've;764
go;753
still;732
see;720
#;716
test;709
thing;702
great;697
vehicle;689
back;688
lot;683
way;672
buy;654
manual;634
feel;628
seat;617
need;615
many;609
model;598
rwd;595
best;595
going;587
3;585
awd;580
--;571
mile;568
take;565
'd;556
may;555
toyota;555
problem;543
point;536
dealer;530
le;528
'll;526
looking;524
tire;524
come;521
right;516
handling;513
%;510
cost;508
since;507
said;505
never;500
road;500
got;497
're;492
power;485
nissan;479
fwd;475
nice;471
sure;468
driver;466
prob

## create a 2-D array to store time of terms co-occurence

In [293]:
from collections import defaultdict
c = defaultdict(dict)
# make a copy dictionary to handle the case of term pair only count once for each post 
copy = defaultdict(dict)

# insert default value 0 into array
for i in brands:
   
    c[i] = 0
# insert default value 0 into array
for z in brands:
    
    copy[i] = 0

In [294]:
import string
filter_word = set(list(string.punctuation) + stopwords.words('english'))

def tokenizer(s):
    tokens = nltk.word_tokenize(s)
    cleanup = [token.lower() for token in tokens if token not in filter_word]
    return cleanup

### total count of brands

In [295]:
brand_count = {}
for brand in brands:
    brand_count[brand] = 0

for sentence in comment2:
    sentence=sentence.strip()
    tokens=[token for token in tokenizer(sentence) if token!=u""]
    for brand in brands:
        if brand in tokens:
            brand_count[brand] += 1  
print brand_count

{'toyata': 0, 'seat': 222, 'subaru': 143, 'buick': 85, 'lexus': 639, 'audi': 747, 'mercury': 5, 'saturn': 7, 'chevrolet': 118, 'suzuki': 20, 'volkswagen': 187, 'dodge': 47, 'pontiac': 58, 'cadillac': 407, 'honda': 489, 'hyundai': 130, 'ford': 183, 'mazda': 124, 'hyundai.': 1, 'infiniti': 1019, 'bmw': 1865, 'hyndai kia': 0, 'sedan': 578, 'kia': 18, 'mitsubishi': 21, 'problem': 240, 'lincoln': 81, 'acura': 1272, 'jaguar': 129, 'nissan': 256, 'toyota': 375, 'volvo': 203, 'chrysler': 72, 'mercedes': 501, 'hyundai,': 0}


###  the total count of "Aspiration"

In [296]:
aspir_count = 0

for sentence in comment2:
    sentence = sentence.strip()
    tokens=[token for token in tokenizer(sentence) if token!=u""]
    if 'aspiration' in tokens:
        aspir_count += 1
print aspir_count


1074


In [297]:
window_size = 10
for sentence in comment2:
    sentence=sentence.strip()
    tokens=[token for token in tokenizer(sentence) if token!=u""]
    for pos,token in enumerate(tokens):
        if token.lower() in brands:
            start=max(0,pos-window_size)
            end=min(len(tokens),pos+window_size+1) 
            for pos2 in xrange(start,end):
                if pos2==pos: 
                    continue
                aspire = tokens[pos2]
                if 'aspiration'in aspire:
                    if c[token] == copy[token]:
                        c[token] = c[token] + 1
                        
           
    copy = c.copy()
c

defaultdict(dict,
            {'acura': 174,
             'audi': 91,
             'bmw': 260,
             'buick': 5,
             'cadillac': 54,
             'chevrolet': 7,
             'chrysler': 4,
             'dodge': 2,
             'ford': 14,
             'honda': 48,
             'hyndai kia': 0,
             'hyundai': 7,
             'hyundai,': 0,
             'hyundai.': 0,
             'infiniti': 131,
             'jaguar': 14,
             'kia': 2,
             'lexus': 75,
             'lincoln': 6,
             'mazda': 12,
             'mercedes': 52,
             'mercury': 0,
             'mitsubishi': 3,
             'nissan': 23,
             'pontiac': 5,
             'problem': 22,
             'saturn': 0,
             'seat': 30,
             'sedan': 63,
             'subaru': 19,
             'suzuki': 1,
             'toyata': 0,
             'toyota': 40,
             'volkswagen': 17,
             'volvo': 22})

## total count of comments


In [298]:
N = len(comment2)
print N

5062


## Calculate Life

In [299]:
lift = {}
for i in brands:
    if brand_count[i] != 0:
        #print i, N, c[i], aspir_count, brand_count[i]
        lift[i] = (N*c[i]*1.0)/(aspir_count*brand_count[i]*1.0)
        print i,lift[i]
   

acura 0.6447331436
audi 0.574167557879
bmw 0.657071107982
buick 0.277248329499
cadillac 0.625341440984
chevrolet 0.279597891614
chrysler 0.261845644527
dodge 0.20056262134
ford 0.360574330169
honda 0.462647519165
hyundai 0.253788855465
hyundai. 0.0
infiniti 0.60591955819
kia 0.523691289054
lincoln 0.349127526036
mazda 0.456118219499
mercedes 0.489196653248
mercury 0.0
mitsubishi 0.673317371641
nissan 0.423453503259
pontiac 0.406312207025
problem 0.43204531347
saturn 0.0
seat 0.636921838039
sedan 0.513724845837
subaru 0.626232240757
suzuki 0.235661080074
toyota 0.502743637492
volkswagen 0.428474691045
volvo 0.510792488831
lexus 0.553195023649
jaguar 0.511512421867
